In [10]:
import sklearn
import matplotlib
import cv2
import os
import glob
import pandas as pd
import numpy as np

In [49]:
cd E:\0804~0822\20220804\AI\크로스핏\프레스\푸쉬프레스\정상\중급\신주호\4

E:\0804~0822\20220804\AI\크로스핏\프레스\푸쉬프레스\정상\중급\신주호\4


In [12]:
err = glob.glob("./**/*.csv",recursive=True)
err = err[1:]

a=[]
max_index=0
for i in range(0,len(err)):
    a.append(pd.read_csv(err[i]))
    if len(a[i])>len(a[max_index]):
        max_index = i
drop_indx =[]
for i in range(0,len(err)):
    for aa in a[max_index]['image_filename'].values:
        if aa not in a[i]['image_filename'].values:
            drop_indx.append(aa)
            
full_indx = a[max_index]['image_filename'].values.tolist()
final = [i for i in full_indx if i not in drop_indx ]

['.\\camera0\\local_keypoints\\Motion2-1 - 4of4.csv',
 '.\\camera1\\local_keypoints\\Motion2-2 - 4of4.csv',
 '.\\camera2\\local_keypoints\\Motion2-3 - 4of4.csv',
 '.\\camera3\\local_keypoints\\Motion2-4 - 4of4.csv',
 '.\\camera4\\local_keypoints\\Motion2-5 - 4of4.csv',
 '.\\camera5\\local_keypoints\\Motion2-6 - 4of4.csv',
 '.\\camera6\\local_keypoints\\Motion2-7 - 4of4.csv',
 '.\\camera7\\local_keypoints\\Motion2-8 - 4of4.csv']

In [20]:
column_idx = ['Nose_x','Nose_y','Nose_z','LEye_x','LEye_y','LEye_z','REye_x','REye_y','REye_z','LEar_x','LEar_y','LEar_z',
           'REar_x','REar_y','REar_z','LShoulder_x','LShoulder_y','LShoulder_z','RShoulder_x','RShoulder_y','RShoulder_z',
           'LElbow_x','LElbow_y','LElbow_z','RElbow_x','RElbow_y','RElbow_z','LWrist_x','LWrist_y','LWrist_z',
           'RWrist_x','RWrist_y','RWrist_z','LHip_x','LHip_y','LHip_z','RHip_x','RHip_y','RHip_z','LKnee_x','LKnee_y','LKnee_z',
           'RKnee_x','RKnee_y','RKnee_z','LAnkle_x','LAnkle_y','LAnkle_z','RAnkle_x','RAnkle_y','RAnkle_z','Head_x','Head_y','Head_z',
           'Neck_x','Neck_y','Neck_z','Hip_x','Hip_y','Hip_z','LBigToe_x','LBigToe_y','LBigToe_z','RBigToe_x','RBigToe_y','RBigToe_z',
           'LSmallToe_x','LSmallToe_y','LSmallToe_z','RSmallToe_x','RSmallToe_y','RSmallToe_z','LHeel_x','LHeel_y','LHeel_z','RHeel_x','RHeel_y','RHeel_z']

K = np.array([[1050.74627,0,959.5],[0,1052.54163,539.5],[0,0,1]])

P = []
Rt1 = np.array([[-0.060679, 0.998130, 0.007343,-605.413346],[0.418569, 0.032123, -0.907617, 548.137858],[-0.906156,-0.052000,-0.419735,4336.592638]])
P.append(np.dot(K,Rt1)) 
Rt2 = np.array([[ -0.066097,-0.997812,0.001260,653.352810],[-0.432199,0.027492,-0.901359,1175.918886],[0.899353 ,-0.060121,-0.433071,3114.943323]])
P.append(np.dot(K,Rt2)) 
Rt3 = np.array([[ 0.784980,-0.619352,-0.014461,-420.172604],[-0.276741,-0.329672,-0.902624,295.490400],[0.554275,0.712544,-0.430186 ,4881.382220]])
P.append(np.dot(K,Rt3)) 
Rt4 = np.array([[-0.999920,-0.012570,-0.001528,728.642753],[-0.003610,0.398618,-0.917110,7.551169],[0.012137,-0.917031,-0.398631,5699.942529]])
P.append(np.dot(K,Rt4)) 
Rt5 = np.array([[-0.733829,-0.677994,-0.042651,1966.137295],[-0.217095,0.293539,-0.930970,76.021013],[0.643711,-0.673913,-0.362597,5872.706559]])
P.append(np.dot(K,Rt5)) 
Rt6 = np.array([[0.799458,0.600171,-0.025722,-704.819635],[0.247259,-0.367782,-0.896437,-127.891999],[-0.547476,0.710304,-0.442424,5625.110129]])
P.append(np.dot(K,Rt6)) 
Rt7 = np.array([[-0.755252,0.655282,0.014137,-737.902102],[0.279717,0.341747,-0.897200,-790.877554],[-0.592750,-0.673657,-0.441399,6746.539080]])
P.append(np.dot(K,Rt7)) 
Rt8 = np.array([[0.999871,0.015922,0.002158,-621.529759],[0.008548,-0.413380,-0.910518,660.889110],[-0.013605,0.910419,-0.413463,4254.063740]])
P.append(np.dot(K,Rt8)) 

def triangulate_nviews(P, ip):
    if not len(ip) == len(P):
        raise ValueError('Number of points and number of cameras not equal.')
    n = len(P)
    M = np.zeros([3*n, 4+n])
    for i, (x, p) in enumerate(zip(ip, P)):
        M[3*i:3*i+3, :4] = p
        M[3*i:3*i+3, 4+i] = -x
    V = np.linalg.svd(M)[-1]
    X = V[-1, :4]
    return X / X[3]

In [ ]:
threed = pd.read_csv('3d_points.csv').to_numpy()
frame = threed.shape[0]
a=[]
for frame_idx in range(frame): #frame 개수
  point_array0=[]
  for point_idx in range(1,27):
    point_array0.append([threed[frame_idx][3*point_idx-2],threed[frame_idx][3*point_idx-1],threed[frame_idx][3*point_idx]])
  a.append(point_array0)
threedarr = np.array(a)

In [26]:
D = []
Pj_cam1 = []
Pj_cam2 = []
Pj_cam3 = []
Pj_cam4 = []
Pj_cam5 = []
Pj_cam6 = []
Pj_cam7 = []
Pj_cam8 = []
for frame_idx in range(frame):
  d = []
  pj1p = []
  pj2p = []
  pj3p = []
  pj4p = []
  pj5p = []
  pj6p = []
  pj7p = []
  pj8p = []
  for point in range(0,26):
    X = threedarr[frame_idx][point]
    X = np.concatenate((X,np.array([1])),axis=0)
    Pjpoint = np.dot(P[0],X)
    Pjpoint /= Pjpoint[2]
    pj1p.append(Pjpoint)

    Pjpoint = np.dot(P[1],X)
    Pjpoint /= Pjpoint[2]
    pj2p.append(Pjpoint)

    Pjpoint = np.dot(P[2],X)
    Pjpoint /= Pjpoint[2]
    pj3p.append(Pjpoint)

    Pjpoint = np.dot(P[3],X)
    Pjpoint /= Pjpoint[2]
    pj4p.append(Pjpoint)

    Pjpoint = np.dot(P[4],X)
    Pjpoint /= Pjpoint[2]
    pj5p.append(Pjpoint)

    Pjpoint = np.dot(P[5],X)
    Pjpoint /= Pjpoint[2]
    pj6p.append(Pjpoint)

    Pjpoint = np.dot(P[6],X)
    Pjpoint /= Pjpoint[2]
    pj7p.append(Pjpoint)

    Pjpoint = np.dot(P[7],X)
    Pjpoint /= Pjpoint[2]
    pj8p.append(Pjpoint)

  D.append(d)
  Pj_cam1.append(pj1p)
  Pj_cam2.append(pj2p)
  Pj_cam3.append(pj3p)
  Pj_cam4.append(pj4p)
  Pj_cam5.append(pj5p)
  Pj_cam6.append(pj6p)
  Pj_cam7.append(pj7p)
  Pj_cam8.append(pj8p)

In [117]:
a = glob.glob("./**/*4.csv",recursive=True)

df_image1 = pd.read_csv(a[0])
if (len(full_indx)-len(drop_indx))<df_image1.shape[0]:
    temp = [df_image1[df_image1['image_filename']==drop].index[0] for drop in drop_indx]
    df_image1 = df_image1.drop(temp,axis=0)
df_image1 = df_image1.drop(['image_filename'],axis=1).to_numpy()

df_image2 = pd.read_csv(a[1])
if (len(full_indx)-len(drop_indx))<df_image2.shape[0]:
    temp = [df_image2[df_image2['image_filename']==drop].index[0] for drop in drop_indx]
    df_image2 = df_image2.drop(temp,axis=0)
df_image2 = df_image2.drop(['image_filename'],axis=1).to_numpy()

df_image3 = pd.read_csv(a[2])
if (len(full_indx)-len(drop_indx))<df_image3.shape[0]:
    temp = [df_image3[df_image3['image_filename']==drop].index[0] for drop in drop_indx]
    df_image3 = df_image3.drop(temp,axis=0)
df_image3 = df_image3.drop(['image_filename'],axis=1).to_numpy()

df_image4 = pd.read_csv(a[3])
if (len(full_indx)-len(drop_indx))<df_image4.shape[0]:
    temp = [df_image4[df_image4['image_filename']==drop].index[0] for drop in drop_indx]
    df_image4 = df_image4.drop(temp,axis=0)
df_image4 = df_image4.drop(['image_filename'],axis=1).to_numpy()

df_image5 = pd.read_csv(a[4])
if (len(full_indx)-len(drop_indx))<df_image5.shape[0]:
    temp = [df_image5[df_image5['image_filename']==drop].index[0] for drop in drop_indx]
    df_image5 = df_image5.drop(temp,axis=0)
df_image5 = df_image5.drop(['image_filename'],axis=1).to_numpy()

df_image6 = pd.read_csv(a[5])
if (len(full_indx)-len(drop_indx))<df_image6.shape[0]:
    temp = [df_image6[df_image6['image_filename']==drop].index[0] for drop in drop_indx]
    df_image6 = df_image6.drop(temp,axis=0)
df_image6 = df_image6.drop(['image_filename'],axis=1).to_numpy()

df_image7 = pd.read_csv(a[6])
if (len(full_indx)-len(drop_indx))<df_image7.shape[0]:
    temp = [df_image7[df_image7['image_filename']==drop].index[0] for drop in drop_indx]
    df_image7 = df_image7.drop(temp,axis=0)
df_image7 = df_image7.drop(['image_filename'],axis=1).to_numpy()

df_image8 = pd.read_csv(a[7])
if (len(full_indx)-len(drop_indx))<df_image8.shape[0]:
    temp = [df_image8[df_image8['image_filename']==drop].index[0] for drop in drop_indx]
    df_image8 = df_image8.drop(temp,axis=0)
df_image8 = df_image8.drop(['image_filename'],axis=1).to_numpy()

In [121]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import math
from sklearn.metrics import mean_squared_error
from cv2 import imshow
R_array = np.array([[3, 17],[4 , 17],[3,18],[4,18],[5,18],[6,18],[5,7],[6,8],[7,9],[8,10],[11,19],[12,19],[11,13],[12,14],[13,15],[14,16],[15,24],[16,25],[20,24],[21,25],[22,24],[23,25],[19,18]])
mmMSE =0
for i,fi in enumerate(final):

  a = glob.glob("./**/"+str(fi),recursive=True)
  fi = int(fi.rstrip('.jpg'))

  frame0 = cv2.imread(a[0])
  frame1 = cv2.imread(a[1])
  frame2 = cv2.imread(a[2])
  frame3 = cv2.imread(a[3])
  frame4 = cv2.imread(a[4])
  frame5 = cv2.imread(a[5])
  frame6 = cv2.imread(a[6])
  frame7 = cv2.imread(a[7])
  distarr2=[]
  for j in range(0,26):
    distarr = []
    cv2.circle(frame0, (int(df_image1[i][2*j]),int(df_image1[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame0, (int(Pj_cam1[i][j][0]),int(Pj_cam1[i][j][1])),7,(0,0,255),thickness=-1)
    cv2.circle(frame1, (int(df_image2[i][2*j]),int(df_image2[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame1, (int(Pj_cam2[i][j][0]),int(Pj_cam2[i][j][1])),7,(0,0,255),thickness=-1)
    cv2.circle(frame2, (int(df_image3[i][2*j]),int(df_image3[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame2, (int(Pj_cam3[i][j][0]),int(Pj_cam3[i][j][1])),7,(0,0,255),thickness=-1)
    cv2.circle(frame3, (int(df_image4[i][2*j]),int(df_image4[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame3, (int(Pj_cam4[i][j][0]),int(Pj_cam4[i][j][1])),7,(0,0,255),thickness=-1)
    cv2.circle(frame4, (int(df_image5[i][2*j]),int(df_image5[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame4, (int(Pj_cam5[i][j][0]),int(Pj_cam5[i][j][1])),7,(0,0,255),thickness=-1)
    cv2.circle(frame5, (int(df_image6[i][2*j]),int(df_image6[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame5, (int(Pj_cam6[i][j][0]),int(Pj_cam6[i][j][1])),7,(0,0,255),thickness=-1)
    cv2.circle(frame6, (int(df_image7[i][2*j]),int(df_image7[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame6, (int(Pj_cam7[i][j][0]),int(Pj_cam7[i][j][1])),7,(0,0,255),thickness=-1)
    cv2.circle(frame7, (int(df_image8[i][2*j]),int(df_image8[i][2*j+1])),7,(0,255,0),thickness=-1)
    cv2.circle(frame7, (int(Pj_cam8[i][j][0]),int(Pj_cam8[i][j][1])),7,(0,0,255),thickness=-1)
    distarr.append(math.sqrt((df_image1[i][2*j]-Pj_cam1[i][j][0])**2  + (df_image1[i][2*j+1]-Pj_cam1[i][j][1])**2))
    distarr.append(math.sqrt((df_image2[i][2*j]-Pj_cam2[i][j][0])**2  +  (df_image2[i][2*j+1]-Pj_cam2[i][j][1])**2))
    distarr.append(math.sqrt((df_image3[i][2*j]-Pj_cam3[i][j][0])**2  +  (df_image3[i][2*j+1]-Pj_cam3[i][j][1])**2))
    distarr.append(math.sqrt((df_image4[i][2*j]-Pj_cam4[i][j][0])**2  +  (df_image4[i][2*j+1]-Pj_cam4[i][j][1])**2))
    distarr.append(math.sqrt((df_image5[i][2*j]-Pj_cam5[i][j][0])**2  +  (df_image5[i][2*j+1]-Pj_cam5[i][j][1])**2))
    distarr.append(math.sqrt((df_image6[i][2*j]-Pj_cam6[i][j][0])**2  +  (df_image6[i][2*j+1]-Pj_cam6[i][j][1])**2))
    distarr.append(math.sqrt((df_image7[i][2*j]-Pj_cam7[i][j][0])**2  +  (df_image7[i][2*j+1]-Pj_cam7[i][j][1])**2))
    distarr.append(math.sqrt((df_image8[i][2*j]-Pj_cam8[i][j][0])**2  +  (df_image8[i][2*j+1]-Pj_cam8[i][j][1])**2))
    distarr2.append(distarr)
  
  distarr2 = np.array(distarr2).T

  mse2=[]
  
  for k in range(0,8):
    mse2.append(mean_squared_error(np.zeros(26),distarr2[k]))
  
  for r_list in R_array:
    a = r_list[0]
    b = r_list[1]
    cv2.line(frame0,(int(df_image1[i][a*2]),int(df_image1[i][a*2+1])),(int(df_image1[i][b*2]),int(df_image1[i][b*2+1])),(0,255,0),2)
    cv2.line(frame0,(int(Pj_cam1[i][a][0]),int(Pj_cam1[i][a][1])),(int(Pj_cam1[i][b][0]),int(Pj_cam1[i][b][1])),(0,0,255),2)
    cv2.line(frame1,(int(df_image2[i][a*2]),int(df_image2[i][a*2+1])),(int(df_image2[i][b*2]),int(df_image2[i][b*2+1])),(0,255,0),2)
    cv2.line(frame1,(int(Pj_cam2[i][a][0]),int(Pj_cam2[i][a][1])),(int(Pj_cam2[i][b][0]),int(Pj_cam2[i][b][1])),(0,0,255),2)
    cv2.line(frame2,(int(df_image3[i][a*2]),int(df_image3[i][a*2+1])),(int(df_image3[i][b*2]),int(df_image3[i][b*2+1])),(0,255,0),2)
    cv2.line(frame2,(int(Pj_cam3[i][a][0]),int(Pj_cam3[i][a][1])),(int(Pj_cam3[i][b][0]),int(Pj_cam3[i][b][1])),(0,0,255),2)
    cv2.line(frame3,(int(df_image4[i][a*2]),int(df_image4[i][a*2+1])),(int(df_image4[i][b*2]),int(df_image4[i][b*2+1])),(0,255,0),2)
    cv2.line(frame3,(int(Pj_cam4[i][a][0]),int(Pj_cam4[i][a][1])),(int(Pj_cam4[i][b][0]),int(Pj_cam4[i][b][1])),(0,0,255),2)
    cv2.line(frame4,(int(df_image5[i][a*2]),int(df_image5[i][a*2+1])),(int(df_image5[i][b*2]),int(df_image5[i][b*2+1])),(0,255,0),2)
    cv2.line(frame4,(int(Pj_cam5[i][a][0]),int(Pj_cam5[i][a][1])),(int(Pj_cam5[i][b][0]),int(Pj_cam5[i][b][1])),(0,0,255),2)
    cv2.line(frame5,(int(df_image6[i][a*2]),int(df_image6[i][a*2+1])),(int(df_image6[i][b*2]),int(df_image6[i][b*2+1])),(0,255,0),2)
    cv2.line(frame5,(int(Pj_cam6[i][a][0]),int(Pj_cam6[i][a][1])),(int(Pj_cam6[i][b][0]),int(Pj_cam6[i][b][1])),(0,0,255),2)
    cv2.line(frame6,(int(df_image7[i][a*2]),int(df_image7[i][a*2+1])),(int(df_image7[i][b*2]),int(df_image7[i][b*2+1])),(0,255,0),2)
    cv2.line(frame6,(int(Pj_cam7[i][a][0]),int(Pj_cam7[i][a][1])),(int(Pj_cam7[i][b][0]),int(Pj_cam7[i][b][1])),(0,0,255),2)
    cv2.line(frame7,(int(df_image8[i][a*2]),int(df_image8[i][a*2+1])),(int(df_image8[i][b*2]),int(df_image8[i][b*2+1])),(0,255,0),2)
    cv2.line(frame7,(int(Pj_cam8[i][a][0]),int(Pj_cam8[i][a][1])),(int(Pj_cam8[i][b][0]),int(Pj_cam8[i][b][1])),(0,0,255),2)

  frame0 = cv2.putText(frame0,"MSE : "+str(np.round(mse2[0],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  frame1 = cv2.putText(frame1,"MSE : "+str(np.round(mse2[1],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  frame2 = cv2.putText(frame2,"MSE : "+str(np.round(mse2[2],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  frame3 = cv2.putText(frame3,"MSE : "+str(np.round(mse2[3],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  frame4 = cv2.putText(frame4,"MSE : "+str(np.round(mse2[4],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  frame5 = cv2.putText(frame5,"MSE : "+str(np.round(mse2[5],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  frame6 = cv2.putText(frame6,"MSE : "+str(np.round(mse2[6],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  frame7 = cv2.putText(frame7,"MSE : "+str(np.round(mse2[7],2)),(800,1000),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
  mmMSE += np.round(np.mean(mse2),2)
  
  frame01 = cv2.hconcat([frame0,frame1])
  frame23 = cv2.hconcat([frame2,frame3])
  frame0123 = cv2.hconcat([frame01,frame23])
  frame45 = cv2.hconcat([frame4,frame5])
  frame67 = cv2.hconcat([frame6,frame7])
  frame4567 = cv2.hconcat([frame45,frame67])
  frame_f = cv2.vconcat([frame0123,frame4567])
  
  if i<10:
    name = './save3/s0000'+str(i)+'.jpg'
    print('0000',i)
  elif i<100:
    name = './save3/s000'+str(i)+'.jpg'
    print('000',i)
  else:
    name = './save3/s00'+str(i)+'.jpg'
    print('00',i)
  cv2.imwrite(name,frame_f)
mmMSE /= frame
print(mmMSE)

0000 0
0000 1
0000 2
0000 3
0000 4
0000 5
0000 6
0000 7
0000 8
0000 9
000 10
000 11
000 12
000 13
000 14
000 15
000 16
000 17
000 18
000 19
000 20
000 21
000 22
000 23
000 24
000 25
000 26
000 27
000 28
000 29
000 30
000 31
000 32
000 33
000 34
000 35
000 36
000 37
000 38
000 39
000 40
000 41
000 42
000 43
000 44
000 45
000 46
000 47
000 48
000 49
000 50
000 51
000 52
000 53
000 54
000 55
000 56
000 57
000 58
000 59
000 60
000 61
000 62
000 63
000 64
000 65
000 66
000 67
000 68
000 69
000 70
000 71
000 72
000 73
000 74
000 75
000 76
000 77
000 78
000 79
000 80
000 81
000 82
000 83
000 84
000 85
000 86
000 87
000 88
000 89
000 90
000 91
000 92
000 93
000 94
000 95
000 96
000 97
000 98
000 99
00 100
00 101
00 102
00 103
00 104
00 105
00 106
00 107
00 108
00 109
00 110
00 111
00 112
00 113
00 114
00 115
00 116
00 117
00 118
00 119
00 120
00 121
00 122
00 123
00 124
00 125
00 126
00 127
00 128
00 129
00 130
00 131
00 132
00 133
00 134
00 135
00 136
00 137
00 138
00 139
00 140
00 141
00 142

In [122]:
%cd save3

E:\0804~0822\20220804\AI\크로스핏\프레스\푸쉬프레스\정상\중급\신주호\4\save3


In [123]:
!cat *.jpg | ffmpeg -f image2pipe -i - output.mkv

ffmpeg version 5.1.2-essentials_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 12.1.0 (Rev2, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb --enable